In [2]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import requests
import requests
from bs4 import BeautifulSoup
import itertools

In [3]:
def set_driver():
    options = uc.ChromeOptions()
    # options.add_argument("--headless")  # Set to True for headless mode
    options.add_argument("--disable-gpu")

    driver = uc.Chrome(options=options)
    # url = "https://life.gocompare.com/life/"
    # url = "https://www.zip-codes.com/m/state/fl.asp"
    # url ="https://www.zip-codes.com/m/state/pr.asp"
    # html = driver.page_source

    # driver.quit()
    # return html
    return driver

In [10]:
def actions(driver, zipcode, mode="physician"):
    if mode == "physician":
        url = "https://www.medicare.gov/care-compare/?providerType=Physician"
    elif mode == "hospitals":
        url = "https://www.medicare.gov/care-compare/?providerType=Hospital"
    driver.get(url)
    driver.set_window_size(928, 1032)
    driver.execute_script("window.scrollTo(0,0)")
    # driver.find_element(By.ID, "mat-input-2").click()
    driver.execute_script("window.scrollTo(0,386)")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(
        (By.XPATH, '//input[@placeholder="Street, ZIP code, city, or state"]'))).send_keys(zipcode)
    # driver.find_element(By.CSS_SELECTOR, ".highlight").click()
    select = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//mat-option/span/span/span"))
    )
    select.click()
    search = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".ProviderSearchSearchButton__submit-text_wrapper"))
    )
    search.click()
    # driver.execute_script("window.scrollTo(0,0)")
    # driver.execute_script("window.scrollTo(0,362)")
    # driver.execute_script("window.scrollTo(0,3992)")
    allclick = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.LINK_TEXT, "All"))
    )
    allclick.click()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ProviderSearchResultCardContainer__details-link')
    links = []
    for div in div_elements:
        try:
            a_tag = div.find_element(By.CSS_SELECTOR, 'a')
            links.append(a_tag.get_attribute('href'))
        except:
            pass

    return links


def extract(links, zipcode):
    names = []
    group_links = []
    addresses_1 = []
    addresses_2 = []
    provider_numbers_1 = []
    provider_numbers_2 = []
    specialties = []
    consetions = []
    genders = []
    educations = []
    urls = []

    i = 1

    for url in links:
        soup = BeautifulSoup(driver.page_source, "html.parser")

        print(f"{i}/{len(links)} | url => {url}")
        urls.append(url)

        driver.get(url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ProviderDetailsHeroContainer__title-name")))

        try:
            name = soup.find('h1', class_='ProviderDetailsHeroContainer__title-name').text.strip()
            names.append(name)
            print(f"Name => {name}")
        except:
            names.append("null")
        # try:
        #     group_link = soup.find('a', class_='ng-star-inserted').text.strip()
        #     group_links.append(group_link)
        #     print(f"Group Link => {group_link}")
        # except:
        #     group_links.append("null")

        # ADDRESSES!!!
        try:
            address_divs = soup.find_all('div', class_='ProviderDetailsLocations__address')
            if len(address_divs) == 2:
                print(f"Address_1 => {address_divs[0].get_text(strip=True)}")
                print(f"Address_2 => {address_divs[1].get_text(strip=True)}")
                addresses_1.append(address_divs[0].get_text(strip=True))
                addresses_2.append(address_divs[1].get_text(strip=True))
            else:
                print(f"Address_1 => {address_divs[0].get_text(strip=True)}")
                addresses_1.append(address_divs[0].get_text(strip=True))
                addresses_2.append("null")
        except:
            addresses_1.append("null")
            addresses_2.append("null")

        # PHONE NUMBERS!!!
        try:
            provider_number_div = soup.find_all('div', class_='ProviderDetailsLocations__phone')
            if len(provider_number_div) == 2:
                print(f"Provider Number_1 => {provider_number_div[0].get_text(strip=True)}")
                print(f"Provider Number_2 => {provider_number_div[1].get_text(strip=True)}")
                provider_numbers_1.append(provider_number_div[0].get_text(strip=True))
                provider_numbers_2.append(provider_number_div[1].get_text(strip=True))
            else:
                print(f"Provider Number_1 => {provider_number_div[0].get_text(strip=True)}")
                provider_numbers_1.append(provider_number_div[0].get_text(strip=True))
                provider_numbers_2.append("null")
        except:
            provider_numbers_1.append("null")
            provider_numbers_2.append("null")
        try:
            specialty = soup.find('div', class_='ProviderDetailsOverview__specialties').text.strip()
            result = specialty.replace("Specialties", "")

            specialties.append(result)
            print(f"Specialty => {result}")
        except:
            specialties.append("null")

        try:
            div_sex_list = soup.find_all('div', text='Sex')
            for div_sex in div_sex_list:
                sex_text = div_sex.find_next_sibling('p').text.strip()
                print(f"Sex => {sex_text}")
            genders.append(sex_text)

        except:
            genders.append("null")

        try:
            education = soup.find('div', text='Education & training').find_next_sibling('p').text.strip()
            educations.append(education)
            print(f"Education => {education}")
        except:
            educations.append("null")

        try:
            consetion = soup.find('div', class_='ProviderDetailsPhysicianHero__additional-info').text.strip()
            consetions.append(consetion)
            print(f"Consetion => {consetion}")
        except:
            consetions.append("null")

        print("----------------------------------------------------------")
        i += 1

    # Create a pandas DataFrame
    data = {
        'name': names,
        'address_1': addresses_1,
        'address_2': addresses_2,
        'provider number 1': provider_numbers_1,
        'provider number 2': provider_numbers_2,
        'specialties': specialties,
        'consetion': consetions,
        'educations': educations,
        'genders': genders,
        'zipcode': zipcode,
        'url': urls
    }

    return data


In [11]:
def extract_hospitals(links, zipcode):
    names = []
    group_links = []
    addresses = []
    provider_numbers = []
    qualities = []
    hospital_types = []
    emergency_services = []
    urls = []

    i = 1

    for url in links:
        soup = BeautifulSoup(driver.page_source, "html.parser")

        print(f"{i}/{len(links)} | url => {url}")
        urls.append(url)

        driver.get(url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ProviderDetailsHeroContainer__title-name")))

        try:
            name = soup.find('h1', class_='ProviderDetailsHeroContainer__title-name').text.strip()
            names.append(name)
            print(f"Name => {name}")
        except:
            names.append("null")

        try:
            group_link = soup.find('a', class_='ng-star-inserted').text.strip()
            group_links.append(group_link)
            print(f"Group Link => {group_link}")
        except:
            group_links.append("null")

        try:
            qualities = soup.find('div', class_='ProviderDetailsMeasuresMenuSection__indicators')
            qualities_texts = qualities.find_all('h3')
            qualities_text = ', '.join(h3.text.strip() for h3 in qualities_texts)
            qualities.append(qualities_text)
            print(f"Qualities => {qualities_text}")
        except:
            qualities.append("null")

        try:
            parent_div = soup.find('div', class_='ProviderDetailsOverviewContainer ProviderDetails__section-container container')
            mat_p2_elements = parent_div.find_all('p', class_='mat-p2')
            if len(mat_p2_elements) >= 2:
                hospital_type = mat_p2_elements[0].text.strip()
                emergency_services = mat_p2_elements[1].text.strip()
                print(f"Hospital Type: {hospital_type}")
                print(f"Emergency Services: {emergency_services}")
                hospital_types.append(hospital_type)
                emergency_services.append(emergency_services)
        except:
            hospital_types.append("null")
            emergency_services.append("null")

        try:
            address_divs = soup.find_all('div', class_='ProviderDetailsLocations__address')
            address = ', '.join([div.get_text(strip=True) for div in address_divs])
            addresses.append(address)
            print(f"Address => {address}")
        except:
            addresses.append("null")
        try:
            provider_number_div = soup.find_all('div', class_='ProviderDetailsLocations__phone')
            provider_number = ', '.join([div.get_text(strip=True) for div in provider_number_div])
            provider_numbers.append(provider_number)
            print(f"Provider number => {provider_number}")
        except:
            provider_numbers.append("null")

        print("----------------------------------------------------------")
        i += 1

    # Create a pandas DataFrame
    data = {
        'name': names,
        'groupLink': group_links,
        'address': addresses,
        'providerNumber': provider_numbers,
        'zipcode': zipcode,
        'qualities': qualities,
        'hospitalTypes': hospital_types,
        'emergencyServices': emergency_services,
        'urls': urls
    }

    return data


In [13]:
zipcode = "00961"

driver = set_driver()
# For Physicians
links = actions(driver, zipcode)
print(links)
print(f"Total Links => {len(links)}")
# time.sleep(60)

data = extract(links, zipcode)
print(data)
print(len(data))

df = pd.DataFrame(data)

# Write the DataFrame to a CSV file
csv_filename = f'provider_data_{zipcode}.csv'
df.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")

driver.quit()

# For Hospitals
# links = actions(driver, zipcode, mode="hospitals")
# print(links)
# print(f"Total Links => {len(links)}")

['https://www.medicare.gov/care-compare/details/physician/1720171663?addressId=PR009616309BA44XXXVEVE300&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1467418376?addressId=PR009616344BA73XXXVEVE400&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1821306127?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1528667342?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1316223241?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1790976405?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3%B3n&state=PR&zipcode=00961', 'https://www.medicare.gov/care-compare/details/physician/1861947137?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3

C:\Users\MAK\AppData\Local\Temp\ipykernel_1256\2859392831.py:122: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  div_sex_list = soup.find_all('div', text='Sex')
C:\Users\MAK\AppData\Local\Temp\ipykernel_1256\2859392831.py:132: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  education = soup.find('div', text='Education & training').find_next_sibling('p').text.strip()


----------------------------------------------------------
2/300 | url => https://www.medicare.gov/care-compare/details/physician/1467418376?addressId=PR009616344BA73XXXVEVE400&city=Bayam%C3%B3n&state=PR&zipcode=00961
Name => Norma A. Feliberti
Address_1 => 44 Dr VeveBayamon, PR 00961-6309
Provider Number_1 => (787) 269-3250
Specialty => Internal medicine
Sex => Female
Education => Graduated,  1981
Consetion => Charges the Medicare-approved amount (so you pay less out-of-pocket)

Telehealth services
----------------------------------------------------------
3/300 | url => https://www.medicare.gov/care-compare/details/physician/1821306127?addressId=PR009600000BAEDIFIBARB401&city=Bayam%C3%B3n&state=PR&zipcode=00961
Name => Luis A. Ortiz Whatts
Address_1 => 73 Calle Dr VeveBayamon, PR 00961-6344
Provider Number_1 => (787) 787-0732
Specialty => Optometry
Sex => Male
Education => Pacific University - College of Optometry,  1968
Consetion => Charges the Medicare-approved amount (so you pay l

In [49]:
data = extract(links, "32004")

b'<!DOCTYPE html>\n<html lang="en-US" dir="ltr" data-critters-container>\n  <head>\n    <script type="text/javascript" src="https://www.medicare.gov/static/e991ccce7893354994b8f762fa69d172d6750013ed7077"  ></script><script>\n      /**\n       *  If using IE, redirect to outdated.\n       *  Should *probably* keep this as the first thing to execute on the page.\n       * */\n      (function (userAgent, href) {\n        var isInternetExplorer =\n          userAgent.indexOf(\'MSIE \') !== -1 ||\n          !!userAgent.match(/Trident.*rv\\:11\\./);\n        if (isInternetExplorer) {\n          var isSpanishSite = href.match(\'/es/\'),\n            outdatedUrl = isSpanishSite\n              ? \'assets/html/outdated.es.html\'\n              : \'assets/html/outdated.html\';\n          window.location.href = outdatedUrl;\n        }\n      })(window.navigator.userAgent, window.location.href);\n    </script>\n    <link rel="preconnect" href="https://fonts.googleapis.com" crossorigin>\n    <link r

C:\Users\Dev\AppData\Local\Temp\ipykernel_4764\535905251.py:88: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  div_sex_list = soup.find_all('div', text='Sex')


b'<!DOCTYPE html>\n<html lang="en-US" dir="ltr" data-critters-container>\n  <head>\n    <script type="text/javascript" src="https://www.medicare.gov/static/e991ccce7893354994b8f762fa69d172d6750013ed7077"  ></script><script>\n      /**\n       *  If using IE, redirect to outdated.\n       *  Should *probably* keep this as the first thing to execute on the page.\n       * */\n      (function (userAgent, href) {\n        var isInternetExplorer =\n          userAgent.indexOf(\'MSIE \') !== -1 ||\n          !!userAgent.match(/Trident.*rv\\:11\\./);\n        if (isInternetExplorer) {\n          var isSpanishSite = href.match(\'/es/\'),\n            outdatedUrl = isSpanishSite\n              ? \'assets/html/outdated.es.html\'\n              : \'assets/html/outdated.html\';\n          window.location.href = outdatedUrl;\n        }\n      })(window.navigator.userAgent, window.location.href);\n    </script>\n    <link rel="preconnect" href="https://fonts.googleapis.com" crossorigin>\n    <link r

In [25]:
def parse_data(html):
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Find all rows in the table
    rows = soup.find_all("tr")
    print(rows)
    # Initialize lists to store data
    zip_codes = []
    cities = []
    counties = []
    types = []

    # Iterate over each row
    for row in rows:
        # Extract data from each cell in the row
        cells = row.find_all("td")

        # Ensure the row has data
        if cells:
            # Extract and append data to respective lists
            try:
                zip_codes.append(cells[0].text)
            except:
                zip_codes.append("null")
            try:
                cities.append(cells[1].text)
            except:
                cities.append("null")
            try:
                counties.append(cells[2].text)
            except:
                counties.append("null")
            try:
                types.append(cells[3].text)
            except:
                types.append("null")

    # Create a Pandas DataFrame
    df = pd.DataFrame({
        "Zip Code": zip_codes,
        "City": cities,
        "County": counties,
        "Type": types
    })

    return df


def save_to_excel(df):
    # Save DataFrame to Excel file
    df.to_excel("zip_code_pertorico.xlsx", index=False)

In [26]:
html = driver()
df = parse_data(html)
save_to_excel(df)

[<tr><td valign="top"><span class="dT">Timezone(s):<span> All Time Zones found within this State. In the United States, there are 7 standard time zones which are, from west to east: Hawaii-Aleutian, Alaska, Pacific, Mountain, Central, Eastern, Atlantic. Also included in U.S. Territories are Samoa, Guam and Palau. The alignment between ZIP codes and time zones is not perfect. A single time zone can cover multiple ZIP codes, and, conversely, a single ZIP code could theoretically span across multiple time zones, although this is quite rare.</span></span></td><td>Atlantic (GMT -04:00)</td></tr>, <tr><td> </td><td></td></tr>, <tr><td><span class="dT">Population:<span> The total Population for Puerto Rico, taken from the 2020 U.S. Census DP1. This population total includes all people residing in Puerto Rico and counts them regardless of citizenship or immigration status. Included are U.S. Citizens, permanent residents, temporary migrants, and undocumented workers. The total population repres

In [ ]:
def fill_form(driver, yesSelection, yearAge):
    try:
        button_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/a"))
        )
        # Click the button
        button_element.click()

    except:
        pass
    #just me selection

    button_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'Just me\')]"))
    )
    # Click the button
    button_element.click()

    # family box selection 
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//label[contains(.,\'My family\')]"))
    )
    search_box.click()

    # age selection 
    dropdown = driver.find_element(By.ID, "coverLength")
    dropdown.find_element(By.XPATH, "//option[. = '5 years']").click()
    # age_box = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.ID, "coverLength")))
    # age_box.click()
    # year = "//option[. = '1 year']"
    # subElementSelection = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.XPATH, year)))
    # subElementSelection.click()

    #coverage amount selection
    coverageAmountSelector = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "coverAmount")))
    actions = ActionChains(driver)
    actions.double_click(coverageAmountSelector).perform()
    actions = ActionChains(driver)

    # Then, we send CTRL+A to select all text
    actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL)

    # Send the BACKSPACE key to delete the selected text
    actions.send_keys(Keys.BACKSPACE)

    # Type in the new value you want to send
    actions.send_keys("50000")

    # Finally, we perform the actions
    actions.perform()
    # coverageAmountSelector.send_keys(ctrl+a)
    # coverageAmountSelector.send_keys(backspace)
    # coverageAmountSelector.send_keys("50000")
    driver.execute_script("window.scrollTo(0,1800)")
    element = driver.find_element(By.ID, "customerTitle")
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    # 19 | click | id=customerTitle | 
    driver.find_element(By.ID, "customerTitle").click()
    # 20 | select | id=customerTitle | label=Mr
    dropdown = driver.find_element(By.ID, "customerTitle")
    dropdown.find_element(By.XPATH, "//option[. = 'Mr']").click()

    driver.find_element(By.XPATH, '//*[@id="forename"]').send_keys("carl")

    driver.find_element(By.XPATH, '//*[@id="surname"]').send_keys("jhonson")
    #static input
    dropdown = driver.find_element(By.ID, "dobDay")
    dropdown.find_element(By.XPATH, "//option[. = '8']").click()

    dropdown = driver.find_element(By.ID, "dobMonth")
    dropdown.find_element(By.XPATH, "//option[. = 'Jul']").click()
    # year modification
    # driver.execute_script("window.scrollTo(0,2800)")
    # driver.find_element(By.CSS_SELECTOR, ".utility-question-class:nth-child(5) > .\\_30Wl1UMbFDaaYo7nu1mRJE").click()
    element = driver.find_element(By.ID, "dobYear")
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

    ageYears = "//option[. = '{}']".format(yearAge)
    dropdown = driver.find_element(By.ID, "dobYear")
    dropdown.find_element(By.XPATH, ageYears).click()

    # yearPointerSelection = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.XPATH, "ageYears")))
    # yearPointerSelection.click()

    # yes no modification
    if yesSelection == "Y":
        element = driver.find_element(By.XPATH, "//span[contains(.,\'Yes\')]")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        yesSelectionClick = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'Yes\')]")))
        yesSelectionClick.click()
    else:
        element = driver.find_element(By.XPATH, "//span[contains(.,\'No\')]")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        yesSelectionClick = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'No\')]")))
        yesSelectionClick.click()

    search_input = driver.find_element(By.ID, "customerAddressSearch")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input).send_keys("22").send_keys(Keys.ENTER).perform()
    driver.implicitly_wait(5)
    time.sleep(3)
    search_input = driver.find_element(By.CSS_SELECTOR, ".search-results:nth-child(1) > span")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, ".search-results:nth-child(1) > span").click()
    time.sleep(3)
    # 60 | click | id=customerEmail | 
    search_input = driver.find_element(By.ID, "customerEmail")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    # driver.find_element(By.ID, "customerEmail").click()
    driver.find_element(By.ID, "customerEmail").send_keys("metaphoricalcreator@gmail.com")
    search_input = driver.find_element(By.ID, "customerPhone")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    # driver.find_element(By.ID, "customerPhone").click()
    driver.find_element(By.ID, "customerPhone").send_keys("01234567891")

    time.sleep(3)
    search_input = driver.find_element(By.XPATH, '//*[@id="getQuotes"]/span[2]')
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.pause(2)
    actions.click(search_input)
    actions.perform()

    return driver#%%
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import requests
import requests
from bs4 import BeautifulSoup
import itertools

In [ ]:
def set_driver():
    options = uc.ChromeOptions()
    # options.add_argument("--headless")  # Set to True for headless mode
    options.add_argument("--disable-gpu")

    driver = uc.Chrome(options=options)
    # url = "https://life.gocompare.com/life/"
    # url = "https://www.zip-codes.com/m/state/fl.asp"
    # url ="https://www.zip-codes.com/m/state/pr.asp"
    # html = driver.page_source

    # driver.quit()
    # return html
    return driver

In [ ]:
def actions(driver, zipcode):
    url = "https://www.medicare.gov/care-compare/?providerType=Physician"
    driver.get(url)
    driver.set_window_size(928, 1032)
    driver.execute_script("window.scrollTo(0,0)")
    # driver.find_element(By.ID, "mat-input-2").click()
    driver.execute_script("window.scrollTo(0,386)")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(
        (By.XPATH, '//input[@placeholder="Street, ZIP code, city, or state"]'))).send_keys(zipcode)
    # driver.find_element(By.CSS_SELECTOR, ".highlight").click()
    select = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//mat-option/span/span/span"))
    )
    select.click()
    search = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".ProviderSearchSearchButton__submit-text_wrapper"))
    )
    search.click()
    # driver.execute_script("window.scrollTo(0,0)")
    # driver.execute_script("window.scrollTo(0,362)")
    # driver.execute_script("window.scrollTo(0,3992)")
    allclick = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.LINK_TEXT, "All"))
    )
    allclick.click()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ProviderSearchResultCardContainer__details-link')
    links = []
    for div in div_elements:
        try:
            a_tag = div.find_element(By.CSS_SELECTOR, 'a')
            links.append(a_tag.get_attribute('href'))
        except:
            pass

    return links


def extract(links, zipcode):
    names = []
    group_links = []
    addresses = []
    provider_numbers = []
    specialties = []
    consetions = []
    genders = []
    educations = []

    i = 1

    for url in links:
        soup = BeautifulSoup(driver.page_source, "html.parser")

        print(f"{i}/{len(links)} | url => {url}")

        driver.get(url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ProviderDetailsHeroContainer__title-name")))

        # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
        # response = requests.get(url, headers=headers)
        # time.sleep(0.2)
        # html_content = response.content
        # print(html_content)
        # soup = BeautifulSoup(html_content, "html.parser")
        try:
            name = soup.find('h1', class_='ProviderDetailsHeroContainer__title-name').text.strip()
            names.append(name)
            print(f"Name => {name}")
        except:
            names.append("null")
        try:
            group_link = soup.find('a', class_='ng-star-inserted').text.strip()
            group_links.append(group_link)
            print(f"Group Link => {group_link}")
        except:
            group_links.append("null")

        try:
            address_divs = soup.find_all('div', class_='ProviderDetailsLocations__address')
            address = ', '.join([div.get_text(strip=True) for div in address_divs])
            addresses.append(address)
            print(f"Address => {address}")
        except:
            addresses.append("null")
        try:
            provider_number_div = soup.find_all('div', class_='ProviderDetailsLocations__phone')
            provider_number = ', '.join([div.get_text(strip=True) for div in provider_number_div])
            provider_numbers.append(provider_number)
            print(f"Provider number => {provider_number}")
        except:
            provider_numbers.append("null")
        try:
            specialty = soup.find('div', class_='ProviderDetailsOverview__specialties').text.strip()
            result = specialty.replace("Specialties", "")

            specialties.append(result)
            print(f"Specialty => {result}")
        except:
            specialties.append("null")

        try:
            div_sex_list = soup.find_all('div', text='Sex')
            for div_sex in div_sex_list:
                sex_text = div_sex.find_next_sibling('p').text.strip()
                print(f"Sex => {sex_text}")
            # gender =
            genders.append(sex_text)

        except:
            genders.append("null")

        try:
            education = soup.find('div', class_='ProviderDetailsOverview__education').text.strip()
            educations.append(education)
            print(f"Education => {education}")
        except:
            educations.append("null")

        try:
            consetion = soup.find('div', class_='ProviderDetailsPhysicianHero__additional-info').text.strip()
            consetions.append(consetion)
            print(f"Consetion => {consetion}")
        except:
            consetions.append("null")

        print("----------------------------------------------------------")
        i += 1

    # Create a pandas DataFrame
    data = {
        'name': names,
        'groupLink': group_links,
        'address': addresses,
        'providerNumber': provider_numbers,
        'specialties': specialties,
        'consetion': consetions,
        'educations': educations,
        'genders': genders,
        'zipcode': zipcode

    }

    return data




In [ ]:
driver = set_driver()
links = actions(driver, "00961")
print(links)
print(f"Total Links => {len(links)}")
# time.sleep(60)

# data = extract(links,"00961")
data = extract_info(links, "00961")
print(data)
print(len(data))


In [ ]:
data = extract(links, "32004")

In [ ]:
def parse_data(html):
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Find all rows in the table
    rows = soup.find_all("tr")
    print(rows)
    # Initialize lists to store data
    zip_codes = []
    cities = []
    counties = []
    types = []

    # Iterate over each row
    for row in rows:
        # Extract data from each cell in the row
        cells = row.find_all("td")

        # Ensure the row has data
        if cells:
            # Extract and append data to respective lists
            try:
                zip_codes.append(cells[0].text)
            except:
                zip_codes.append("null")
            try:
                cities.append(cells[1].text)
            except:
                cities.append("null")
            try:
                counties.append(cells[2].text)
            except:
                counties.append("null")
            try:
                types.append(cells[3].text)
            except:
                types.append("null")

    # Create a Pandas DataFrame
    df = pd.DataFrame({
        "Zip Code": zip_codes,
        "City": cities,
        "County": counties,
        "Type": types
    })

    return df


def save_to_excel(df):
    # Save DataFrame to Excel file
    df.to_excel("zip_code_pertorico.xlsx", index=False)

In [ ]:
html = driver()
df = parse_data(html)
save_to_excel(df)

In [ ]:
def fill_form(driver, yesSelection, yearAge):
    try:
        button_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/a"))
        )
        # Click the button
        button_element.click()

    except:
        pass
    #just me selection

    button_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'Just me\')]"))
    )
    # Click the button
    button_element.click()

    # family box selection
    search_box = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//label[contains(.,\'My family\')]"))
    )
    search_box.click()

    # age selection
    dropdown = driver.find_element(By.ID, "coverLength")
    dropdown.find_element(By.XPATH, "//option[. = '5 years']").click()
    # age_box = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.ID, "coverLength")))
    # age_box.click()
    # year = "//option[. = '1 year']"
    # subElementSelection = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.XPATH, year)))
    # subElementSelection.click()

    #coverage amount selection
    coverageAmountSelector = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "coverAmount")))
    actions = ActionChains(driver)
    actions.double_click(coverageAmountSelector).perform()
    actions = ActionChains(driver)

    # Then, we send CTRL+A to select all text
    actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL)

    # Send the BACKSPACE key to delete the selected text
    actions.send_keys(Keys.BACKSPACE)

    # Type in the new value you want to send
    actions.send_keys("50000")

    # Finally, we perform the actions
    actions.perform()
    # coverageAmountSelector.send_keys(ctrl+a)
    # coverageAmountSelector.send_keys(backspace)
    # coverageAmountSelector.send_keys("50000")
    driver.execute_script("window.scrollTo(0,1800)")
    element = driver.find_element(By.ID, "customerTitle")
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    # 19 | click | id=customerTitle |
    driver.find_element(By.ID, "customerTitle").click()
    # 20 | select | id=customerTitle | label=Mr
    dropdown = driver.find_element(By.ID, "customerTitle")
    dropdown.find_element(By.XPATH, "//option[. = 'Mr']").click()

    driver.find_element(By.XPATH, '//*[@id="forename"]').send_keys("carl")

    driver.find_element(By.XPATH, '//*[@id="surname"]').send_keys("jhonson")
    #static input
    dropdown = driver.find_element(By.ID, "dobDay")
    dropdown.find_element(By.XPATH, "//option[. = '8']").click()

    dropdown = driver.find_element(By.ID, "dobMonth")
    dropdown.find_element(By.XPATH, "//option[. = 'Jul']").click()
    # year modification
    # driver.execute_script("window.scrollTo(0,2800)")
    # driver.find_element(By.CSS_SELECTOR, ".utility-question-class:nth-child(5) > .\\_30Wl1UMbFDaaYo7nu1mRJE").click()
    element = driver.find_element(By.ID, "dobYear")
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

    ageYears = "//option[. = '{}']".format(yearAge)
    dropdown = driver.find_element(By.ID, "dobYear")
    dropdown.find_element(By.XPATH, ageYears).click()

    # yearPointerSelection = WebDriverWait(driver, 20).until(
    # EC.presence_of_element_located((By.XPATH, "ageYears")))
    # yearPointerSelection.click()

    # yes no modification
    if yesSelection == "Y":
        element = driver.find_element(By.XPATH, "//span[contains(.,\'Yes\')]")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        yesSelectionClick = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'Yes\')]")))
        yesSelectionClick.click()
    else:
        element = driver.find_element(By.XPATH, "//span[contains(.,\'No\')]")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        yesSelectionClick = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(.,\'No\')]")))
        yesSelectionClick.click()

    search_input = driver.find_element(By.ID, "customerAddressSearch")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input).send_keys("22").send_keys(Keys.ENTER).perform()
    driver.implicitly_wait(5)
    time.sleep(3)
    search_input = driver.find_element(By.CSS_SELECTOR, ".search-results:nth-child(1) > span")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    driver.implicitly_wait(5)
    driver.find_element(By.CSS_SELECTOR, ".search-results:nth-child(1) > span").click()
    time.sleep(3)
    # 60 | click | id=customerEmail |
    search_input = driver.find_element(By.ID, "customerEmail")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    # driver.find_element(By.ID, "customerEmail").click()
    driver.find_element(By.ID, "customerEmail").send_keys("metaphoricalcreator@gmail.com")
    search_input = driver.find_element(By.ID, "customerPhone")
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.click(search_input)
    # driver.find_element(By.ID, "customerPhone").click()
    driver.find_element(By.ID, "customerPhone").send_keys("01234567891")

    time.sleep(3)
    search_input = driver.find_element(By.XPATH, '//*[@id="getQuotes"]/span[2]')
    actions = ActionChains(driver)
    actions.move_to_element(search_input).perform()
    actions.pause(2)
    actions.click(search_input)
    actions.perform()

    return driver


In [ ]:
def second_page_alterations(driver,internalSelection,amount,yearChange):
    try:
        yearPointerSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mobile-down-arrow"]/u')))
        yearPointerSelection.click()
        yearPointerSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]')))
        yearPointerSelection.click()
        if internalSelection == "D":
            yearPointerSelection = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]/option[2]')))
            yearPointerSelection.click()
        else:
            yearPointerSelection = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]/option[3]')))
            yearPointerSelection.click()

            # //*[@id="resultsCoverAmount"]
        yearChange = "//option[. = '{}']".format(yearChange)
        print(yearChange)
        coverageAmountSelector=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "resultsCoverAmount")))
        actions = ActionChains(driver)
        actions.double_click(coverageAmountSelector).perform()
        actions = ActionChains(driver)

        # Then, we send CTRL+A to select all text
        actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL)

        # Send the BACKSPACE key to delete the selected text
        actions.send_keys(Keys.BACKSPACE)

        # Type in the new value you want to send
        actions.send_keys(amount)


        # Finally, we perform the actions
        actions.perform()

        dropdown = driver.find_element(By.ID, "resultsCoverTerm")
        dropdown.find_element(By.XPATH,yearChange).click()

        # driver.find_element(By.XPATH,'//*[@id="resultsCoverAmount"]').send_keys(amount)
        titleSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="updateResults"]/span[1]')))
        titleSelection.click()
    except:
        titleSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div/ul/li[1]/a')))
        titleSelection.click()

    return driver


In [ ]:
def extract_info(driver,internalSelection, amount, yearChange):
    data_list = []

    try:
        WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="nstest"]/div/div/div[2]/div[1]/div[2]/div[1]/div/div[1]')))
    except:
        return data_list

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    life_rows = soup.find_all('div', class_='life-row')


    for row in life_rows:
        # Initialize an empty dictionary for each row
        data_dict = {}
        if internalSelection == "D":
            data_dict['Cover type:'] ="Decreasing"
        else:
            data_dict['Cover type:'] ="Level term"

        data_dict['Cover amount:'] = amount
        data_dict['Cover term (years):'] = yearChange
        # Extract the logo image src
        img_tag = row.find('img', class_='company-logo')
        data_dict['logo_src'] = img_tag['src'] if img_tag else 'No image found'

        # Extract the cover amount
        cover_amount = row.find('div', class_='results-col-sum').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['cover_amount'] = cover_amount

        # Extract the claims paid percentage
        claims_paid = row.find('div', class_='results-col-claims').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['claims_paid'] = claims_paid

        # Extract the premium per month
        premium_per_month = row.find('div', class_='results-col-premium').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['premium_per_month'] = premium_per_month

        # Add the dictionary to the list
        data_list.append(data_dict)

    return data_list

In [ ]:
driver = driver()
#input settings
while True:
    yearAge =input("\n Dynamic input initialisation \n\t input age years like eg:1980 for plans related to it ")
    smokerStatus = input("\n Dynamic input initialisation \n\t  Smoker Status for that year Y for yes N for no ")
    if yearAge.isdigit() and (smokerStatus != 'Y' or smokerStatus != 'N'):
        break
    else:
        print("Invalid input")

fill_form(driver,smokerStatus,yearAge)


In [ ]:

internal_selection_options = ['D', 'Not D']
amount_options = [50000, 75000, 100000, 150000, 160000, 175000, 200000, 250000, 300000, 350000, 400000, 500000]
year_change_options = range(5, 7)  # 1 to 35

combinations = itertools.product(internal_selection_options, amount_options, year_change_options)

# List to store the results
results = []

# Iterate over the combinations and call second_page_alterations for each
for internalSelection, amount, yearChange in combinations:
    print(f"{internalSelection} ,{amount} ,{yearChange}")
    second_page_alterations(driver, internalSelection, amount, yearChange)
    result = extract_info(driver,internalSelection, amount, yearChange)
    results.extend(result)


# Convert the results list to a DataFrame
df = pd.DataFrame(results)
df["SmokerStatus"]="Y"
# Export the DataFrame to an Excel sheet
fileExportName ="quote{}SmokeStatus{}Results.xlsx".format(1960,"Y")

df.to_excel(fileExportName, index=False)

In [10]:
def second_page_alterations(driver,internalSelection,amount,yearChange):
    try:
        yearPointerSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mobile-down-arrow"]/u')))
        yearPointerSelection.click()
        yearPointerSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]')))
        yearPointerSelection.click()
        if internalSelection == "D":
            yearPointerSelection = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]/option[2]')))
            yearPointerSelection.click()
        else:
            yearPointerSelection = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="resultsCoverType"]/option[3]')))
            yearPointerSelection.click()    
        
            # //*[@id="resultsCoverAmount"]
        yearChange = "//option[. = '{}']".format(yearChange)
        print(yearChange)
        coverageAmountSelector=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "resultsCoverAmount")))
        actions = ActionChains(driver)
        actions.double_click(coverageAmountSelector).perform()
        actions = ActionChains(driver)

        # Then, we send CTRL+A to select all text
        actions.key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL)

        # Send the BACKSPACE key to delete the selected text
        actions.send_keys(Keys.BACKSPACE)

        # Type in the new value you want to send
        actions.send_keys(amount)


        # Finally, we perform the actions
        actions.perform()
        
        dropdown = driver.find_element(By.ID, "resultsCoverTerm")
        dropdown.find_element(By.XPATH,yearChange).click()
        
        # driver.find_element(By.XPATH,'//*[@id="resultsCoverAmount"]').send_keys(amount)    
        titleSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="updateResults"]/span[1]')))
        titleSelection.click()
    except:
        titleSelection = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div/ul/li[1]/a')))
        titleSelection.click()
        
    return driver
   

In [11]:
def extract_info(driver,internalSelection, amount, yearChange):
    data_list = []

    try:
        WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="nstest"]/div/div/div[2]/div[1]/div[2]/div[1]/div/div[1]')))
    except:
        return data_list
        
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    life_rows = soup.find_all('div', class_='life-row')


    for row in life_rows:
        # Initialize an empty dictionary for each row
        data_dict = {}
        if internalSelection == "D":
            data_dict['Cover type:'] ="Decreasing"
        else:
            data_dict['Cover type:'] ="Level term"  
            
        data_dict['Cover amount:'] = amount 
        data_dict['Cover term (years):'] = yearChange   
        # Extract the logo image src
        img_tag = row.find('img', class_='company-logo')
        data_dict['logo_src'] = img_tag['src'] if img_tag else 'No image found'
        
        # Extract the cover amount
        cover_amount = row.find('div', class_='results-col-sum').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['cover_amount'] = cover_amount
        
        # Extract the claims paid percentage
        claims_paid = row.find('div', class_='results-col-claims').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['claims_paid'] = claims_paid
        
        # Extract the premium per month
        premium_per_month = row.find('div', class_='results-col-premium').find('span', class_='text-3xl').get_text(strip=True)
        data_dict['premium_per_month'] = premium_per_month
        
        # Add the dictionary to the list
        data_list.append(data_dict)

    return data_list

In [6]:
driver = driver()
#input settings 
while True:
    yearAge =input("\n Dynamic input initialisation \n\t input age years like eg:1980 for plans related to it ")
    smokerStatus = input("\n Dynamic input initialisation \n\t  Smoker Status for that year Y for yes N for no ")
    if yearAge.isdigit() and (smokerStatus != 'Y' or smokerStatus != 'N'):
        break
    else:
        print("Invalid input")

fill_form(driver,smokerStatus,yearAge)


<undetected_chromedriver.Chrome (session="8f5125380364b060cac9b99eaf830353")>

In [12]:

internal_selection_options = ['D', 'Not D']
amount_options = [50000, 75000, 100000, 150000, 160000, 175000, 200000, 250000, 300000, 350000, 400000, 500000]
year_change_options = range(5, 7)  # 1 to 35

combinations = itertools.product(internal_selection_options, amount_options, year_change_options)

# List to store the results
results = []

# Iterate over the combinations and call second_page_alterations for each
for internalSelection, amount, yearChange in combinations:
    print(f"{internalSelection} ,{amount} ,{yearChange}")
    second_page_alterations(driver, internalSelection, amount, yearChange)
    result = extract_info(driver,internalSelection, amount, yearChange)
    results.extend(result)


# Convert the results list to a DataFrame
df = pd.DataFrame(results)
df["SmokerStatus"]="Y"
# Export the DataFrame to an Excel sheet
fileExportName ="quote{}SmokeStatus{}Results.xlsx".format(1960,"Y")

df.to_excel(fileExportName, index=False)


D ,50000 ,5
//option[. = '5']
D ,50000 ,6
//option[. = '6']
D ,75000 ,5
//option[. = '5']
D ,75000 ,6
//option[. = '6']
D ,100000 ,5
//option[. = '5']
D ,100000 ,6
//option[. = '6']
D ,150000 ,5
D ,150000 ,6
//option[. = '6']
D ,160000 ,5
D ,160000 ,6
//option[. = '6']
D ,175000 ,5
//option[. = '5']
D ,175000 ,6
//option[. = '6']
D ,200000 ,5
//option[. = '5']
D ,200000 ,6
//option[. = '6']
D ,250000 ,5
D ,250000 ,6
//option[. = '6']
D ,300000 ,5
D ,300000 ,6
//option[. = '6']
D ,350000 ,5
//option[. = '5']
D ,350000 ,6
//option[. = '6']
D ,400000 ,5
//option[. = '5']
D ,400000 ,6
//option[. = '6']
D ,500000 ,5
D ,500000 ,6
//option[. = '6']
Not D ,50000 ,5
Not D ,50000 ,6
//option[. = '6']
Not D ,75000 ,5
//option[. = '5']
Not D ,75000 ,6
//option[. = '6']
Not D ,100000 ,5
//option[. = '5']
Not D ,100000 ,6
//option[. = '6']
Not D ,150000 ,5
Not D ,150000 ,6
//option[. = '6']
Not D ,160000 ,5
Not D ,160000 ,6
//option[. = '6']
Not D ,175000 ,5
//option[. = '5']
Not D ,175000 ,6
//opti